# Purpose


Purpose of this analysis is to infer market bottoms from Chicago Financial Conditions sub index-risk. 
The National Financial Conditions Index (NFCI) and adjusted NFCI (ANFCI) are each constructed to have an average value of zero and a standard deviation of one over a sample period extending back to 1971. Positive values of the NFCI have been historically associated with tighter-than-average financial conditions, while negative values have been historically associated with looser-than-average financial conditions.
Analysis looks for the dates when NFCI Risk sub index signals tighter-than-average financial conditions compared to previous weeks consecutively i.e. 4 weeks in a row
NFCI Risk sub index includes Financial indicators that are informative on money markets, short term risk on/off sentiments and more importantly it does not include US equity markets. This is important for avoiding endonegeity. The same analysis can be carried out with NFCI Credit sub index 



In [ ]:
import pandas as pd
import string
import numpy as np
import random
import matplotlib.pyplot as plt 
import seaborn as sns
from pandas.tools.plotting import scatter_matrix 
from scipy.stats import f_oneway, ttest_ind
import warnings
warnings.filterwarnings('ignore') 
from IPython.display import display
import random; random.seed(0)
import string
import os
import pandas_datareader.data as web
from pandas.tseries.offsets import *

%matplotlib inline

path='/Users/aybarsatalay/Desktop/Python'
os.chdir(path

In [23]:
start='2009-08-08'
end='2018-02-15'
df=(web.DataReader('NFCIRISK','fred',start,end))
df['daily_return']=df.NFCIRISK.pct_change()
df['neg_returns'] = df.daily_return.lt(0)
df['succesive_downs']=df['neg_returns'].groupby((df['neg_returns']!=df['neg_returns'].shift()).cumsum()).cumsum()
trigger_dates=(df[df['succesive_downs']==4.0].index)
print(trigger_dates)

DatetimeIndex(['2009-09-11', '2010-05-07', '2010-11-19', '2011-06-10',
               '2012-05-04', '2013-06-07', '2014-09-19', '2014-12-26',
               '2015-08-21', '2015-12-18', '2018-02-02'],
              dtype='datetime64[ns]', name='DATE', freq=None)


In [24]:
#get historical data for SPY

spy = pd.read_csv('SPY.csv',index_col=['Date'],parse_dates=True)[['Adj Close']]
spy=pd.DataFrame(spy,index=spy.index)
spy['returns']=spy['Adj Close'].pct_change()
del spy['Adj Close']
spy=spy.dropna()

In [25]:
# In order to quantify if identifying trigger dates produces meaningful results, 
#I've calculated the subsequent cumulative return over N number of days following each instance of the consecutive tightining
#I then calculated the average cumulative return for each N day look ahead period
# The lookahead periods I chose were 1, 2, 3, 5, 10, and 21 days. 
#As can ben seen from the results, there is a clear pattern: Current average cumulative returns are negative across all look ahead period and magnitute of this negativity increases with the range of lookahead

def _cumlrets_given_trigger(trigger_dates, look):
    '''function to calculate conditional cumulative returns given trigger dates and lookahead period'''
    cr = {}
    for date in trigger_dates:
        start_int = spy.index.get_loc(date)
        start = spy.index[start_int]
        end = start + look * BDay()
        cumlrets = spy.ix[start:end].cumsum().iloc[-1]
        cr[date] = cumlrets.returns 
    conditional_rets = pd.Series(cr, index=cr.keys(), \
                                 name='_{} days lookahead_'.format(look)).sort_index()
    return conditional_rets

def _get_avgrets_bylook(trigger_dates):
    '''function to aggregate conditional cumulative returns given trigger dates and lookahead period'''
    lookahead = [1, 2, 3, 5, 10, 21]
    rets_dict = {}
    avg_returns_looks = {} 
    for look in lookahead:
        rets = _cumlrets_given_trigger(trigger_dates, look)
        mean_rets = rets.mean()
        avg_returns_looks['{}_Days'.format(look)] = mean_rets
        rets_dict['{}_Days'.format(look)] = rets
    #print(rets_dict)

    avg_rets_looks = pd.Series(avg_returns_looks, index=avg_returns_looks.keys(),\
                               name='_avg_returns_given_lookahead_')
    avg_rets_looks = avg_rets_looks[['{}_Days'.format(str(i)) for i in lookahead]]
    return avg_rets_looks, rets_dict   
    
"""looking at the average"""
avg_crets, avgdict = _get_avgrets_bylook(trigger_dates)
print (avg_crets)

1_Days    -0.011768
2_Days    -0.012616
3_Days    -0.007475
5_Days    -0.011049
10_Days   -0.023222
21_Days   -0.023482
Name: _avg_returns_given_lookahead_, dtype: float64


In [19]:
#To show distribution (not just averages) below function depicts all trigger dates with selected look ahead periods 
def create_df(trigger_dates):
    lookahead = [1, 2, 3, 5, 10, 21]
    df=pd.DataFrame()
    for look in lookahead:
        rets = _cumlrets_given_trigger(trigger_dates, look)
        #print(rets)
        df=df.append(rets,ignore_index=False)
    
    return (df.T)
df= create_df(trigger_dates)
print(df)

            _1 days lookahead_  _2 days lookahead_  _3 days lookahead_  \
2009-09-11            0.004677            0.008856            0.023991   
2010-05-07            0.029166            0.026325            0.040311   
2010-11-19            0.001919           -0.012557            0.002217   
2011-06-10           -0.013127           -0.000441           -0.018226   
2012-05-04           -0.015428           -0.019440           -0.025371   
2013-06-07            0.012721            0.002405           -0.005872   
2014-09-19           -0.008623           -0.014347           -0.006520   
2014-12-26            0.004568           -0.000798           -0.010721   
2015-08-21           -0.072157           -0.084397           -0.044606   
2015-12-18           -0.009568           -0.000494            0.011890   
2018-02-02           -0.063592           -0.043890           -0.049315   

            _5 days lookahead_  _10 days lookahead_  _21 days lookahead_  
2009-09-11            0.023137      